In [6]:
from item import *
import random
import numpy as np
import scipy

random.seed()

In [ ]:
class NetlistTransformer:
    def __init__(self) -> None:
        self.ratio_condense: float = None
        self.num_nodes: int = None
        self.num_movable_nodes: int = None
        self.num_fixed_nodes: int = None
        self.num_edges: int = None
        self.num_pins: int = None
        self.movable_node_list: list[Node] = []
        self.fixed_node_list: list[Node] = []
        self.edge_list: list[Edge] = []
        self.supernode_list: list[SuperNode] = []
        self.superedge_list: list[SuperEdge] = []
        self.name2node: dict[str, Node] = {}
        self.width_die: int = None
        self.height_die: int = None
        self.die_boundary_x: tuple[int, int] = ()   # (left, right)
        self.die_boundary_y: tuple[int, int] = ()   # (bottom, top)
        self.num_rows: int = None
        self.height_row: int = None
        self.area_movable: int = None
        self.area_available: int = None

        # vectors or matrices
        self.x_nodes: np.ndarray = None             # 1D array, R^N
        self.y_nodes: np.ndarray = None             # 1D array, R^N
        self.adjacency_matrix: np.ndarray = None    # 2D array, R^{N*N}
    
    def readFromTxt(self):
        pass

    def readFromBookshelf(self, circuitName = 'adaptec1', inputPath = '../benchmarks/'):
        # read from .nodes / .pl / .nets / .scl files

        def fileName(fileFormat: str) -> str:
            return inputPath + circuitName + '/' + circuitName + fileFormat
        
        # read .nodes file
        with open(fileName('.nodes'), 'r') as file:
            for line in file:
                words = line.split()
                if words[:1] == ['NumNodes']:
                    self.num_nodes = int(words[-1])
                    break
            for line in file:
                words = line.split()
                if words[:1] == ['NumTerminals']:
                    self.num_fixed_nodes = int(words[-1])
                    break
            self.num_movable_nodes = self.num_nodes - self.num_fixed_nodes

            for line in file:
                words = line.split()
                if len(words) == 3:
                    node = Node(len(self.movable_node_list))
                    node.name = words[0]
                    node.w = int(words[1])
                    node.h = int(words[2])
                    self.movable_node_list.append(node)
                    self.name2node[node.name] = node
                else:
                    node = Node(len(self.fixed_node_list))
                    node.name = words[0]
                    node.w = int(words[1])
                    node.h = int(words[2])
                    self.fixed_node_list.append(node)
                    self.name2node[node.name] = node

        # read .pl file
        with open(fileName('.pl'), 'r') as file:
            orientations = {'N', 'S', 'E', 'W', 'FN', 'FS', 'FE', 'FW'}
            for line in file:
                words = line.split()
                if len(words) >= 5 and words[3] == ':' and words[4] in orientations:
                    node = self.name2node[words[0]]
                    node.x = float(words[1])
                    node.y = float(words[2])
            for line in file:
                words = line.split()
                node = self.name2node[words[0]]
                node.x = float(words[1])
                node.y = float(words[2])

        # read .nets file
        with open(fileName('.nets'), 'r') as file:
            for line in file:
                words = line.split()
                if words[:1] == ['NumNets']:
                    self.num_edges = int(words[-1])
                    break
            for line in file:
                words = line.split()
                if words[:1] == ['NumPins']:
                    self.num_pins = int(words[-1])
                    break
            for line in file:
                words = line.split()
                if words[:1] == ['NetDegree']:
                    edge = Edge(len(self.edge_list))
                    edge.name = words[-1]
                    edge.degree = int(words[-2])
                    for i in range(edge.degree):
                        line = file.readline()
                        words = line.split()
                        node = self.name2node[words[0]]
                        x_offset = float(words[-2])
                        y_offset = float(words[-1])
                        if words[1] == 'I':
                            edge.in_pin_list.append((node, x_offset, y_offset))
                        else:
                            edge.out_pin_list.append((node, x_offset, y_offset))
                        edge.node_set.add(node)
                        node.edge_set.add(edge)
        
        # read .scl file
        with open(fileName('.scl'), 'r') as file:
            for line in file:
                words = line.split()
                if words[:1] == ['NumRows']:
                    self.num_rows = int(words[-1])
                    break
            
            y_min = float('inf')
            for line in file:
                words = line.split()
                if words[:1] == ['CoreRow']:
                    line = file.readline()
                    y_min = min(y_min, int(line.split()[-1]))
                    line = file.readline()
                    self.height_row = int(line.split()[-1])
                    line = file.readline()
                    sitewidth = int(line.split()[-1])
                    line = file.readline()
                    sitespacing = int(line.split()[-1])
                    line = file.readline()
                    line = file.readline()
                    line = file.readline()
                    words = line.split()
                    subrowOrigin = int(words[2])
                    numSites = int(words[5])
                    left = subrowOrigin
                    right = subrowOrigin + (numSites - 1) * sitespacing + sitewidth
                    self.die_boundary_x = (left, right)
            bottom = int(y_min)
            top = int(y_min) + self.height_row * self.num_rows
            self.die_boundary_y = (bottom, top)
            self.width_die = self.die_boundary_x[1] - self.die_boundary_x[0]
            self.height_die = self.die_boundary_y[1] - self.die_boundary_y[0]

    def cluster(self, iteration: int = 10):
        def randomPoint() -> tuple[float, float]:
            (x_low, x_high) = self.die_boundary_x
            (y_low, y_high) = self.die_boundary_y
            x = random.uniform(x_low, x_high)
            y = random.uniform(y_low, y_high)
            return (x,y)
        
        # randomly place
        for node in self.movable_node_list:
            node.setCenter(randomPoint())
        
        # pull force
        for i in range(iteration):
            print(i)
            nodes = random.sample(self.movable_node_list, len(self.movable_node_list))
            for node in nodes:
                node.pull()
        


    def dumpFiles(self):
        pass

    def buildGraph(self):
        pass

In [ ]:
transformer = NetlistTransformer()
transformer.readFromBookshelf()
transformer.cluster(10)